In [1]:
import sys
sys.path.append("..")
from ctdi_treatment.env_setup_start import *
import time
license_keys = set_envvars('/home/ubuntu/hasham/jsl_keys.json')
spark = start_sparknlp(license_keys)

Desired SparkNLP Version: 3.1.3
Desired SparkNLP-JSL Version: 3.1.3
Real Spark NLP Version : 3.1.1
Real Spark NLP_JSL Version : 3.1.3


In [2]:
import pandas as pd
alldf = pd.read_csv("../data/raw_aac.csv")
print (alldf.shape)
print (alldf.columns)
alldf.head()

(45, 12)
Index(['design_group_id', 'nct_id', 'group_type', 'title', 'arm_desc',
       'intervention_id', 'intervention_type', 'name', 'intervention_desc',
       'arm_title_desc', 'intervention_name_desc',
       'intervention_type_name_desc'],
      dtype='object')


,design_group_id,nct_id,group_type,title,arm_desc,intervention_id,intervention_type,name,intervention_desc,arm_title_desc,intervention_name_desc,intervention_type_name_desc
0,13299803,NCT00000435,Placebo Comparator,A,Subjects randomized to arm A received 25mg/day...,12910066,Drug,None-placebo,placebo was taken in pill form at 25mg/day for...,A\n\nSubjects randomized to arm A received 25m...,None-placebo: placebo was taken in pill form a...,Drug\nNone-placebo: placebo was taken in pill ...
1,13299804,NCT00000435,Active Comparator,B,Subjects randomized to Arm B received 25mg/day...,12910065,Drug,dnaJ peptide,dnaJP1 was taken in pill form at 25mg/day for ...,B\n\nSubjects randomized to Arm B received 25m...,dnaJ peptide: dnaJP1 was taken in pill form at...,Drug\ndnaJ peptide: dnaJP1 was taken in pill f...
2,13690845,NCT00001337,Experimental,Arm A,EPOCH + Rituximab every 3 weeks for 6 cycles.,13274570,Drug,EPOCH,Combination chemotherapy given with Rituximab ...,Arm A\n\nEPOCH + Rituximab every 3 weeks for 6...,EPOCH: Combination chemotherapy given with Rit...,Drug\nEPOCH: Combination chemotherapy given wi...
3,13690845,NCT00001337,Experimental,Arm A,EPOCH + Rituximab every 3 weeks for 6 cycles.,13274571,Biological,Rituximab,Rituximab given on Day 1 of combination chemot...,Arm A\n\nEPOCH + Rituximab every 3 weeks for 6...,Rituximab: Rituximab given on Day 1 of combina...,Biological\nRituximab: Rituximab given on Day ...
4,13732309,NCT00001563,Experimental,1,EPOCH-R every 3 weeks for up to 6 cycle,13311178,Biological,Filgrastim,Filgrastim after EPOCH-R from Day 6 for 10 day...,1\n\nEPOCH-R every 3 weeks for up to 6 cycle,Filgrastim: Filgrastim after EPOCH-R from Day ...,Biological\nFilgrastim: Filgrastim after EPOCH...


In [3]:
name_sep = ": "
sent_sep = "."
tit_sep = "\n\n"

# Join title and arm description
alldf["arm_title_desc"] = alldf["title"]+tit_sep+alldf["arm_desc"]
# Join name and intervention desc
alldf["intervention_name_desc"] = alldf["name"]+name_sep+alldf["intervention_desc"]+sent_sep
# Join intervention type and name and intervention desc
#alldf["intervention_type_name_desc"] = alldf["intervention_type"]+tit_sep+alldf["intervention_name_desc"]
alldf.head()

,design_group_id,nct_id,group_type,title,arm_desc,intervention_id,intervention_type,name,intervention_desc,arm_title_desc,intervention_name_desc,intervention_type_name_desc
0,13299803,NCT00000435,Placebo Comparator,A,Subjects randomized to arm A received 25mg/day...,12910066,Drug,None-placebo,placebo was taken in pill form at 25mg/day for...,A\n\nSubjects randomized to arm A received 25m...,None-placebo: placebo was taken in pill form a...,Drug\nNone-placebo: placebo was taken in pill ...
1,13299804,NCT00000435,Active Comparator,B,Subjects randomized to Arm B received 25mg/day...,12910065,Drug,dnaJ peptide,dnaJP1 was taken in pill form at 25mg/day for ...,B\n\nSubjects randomized to Arm B received 25m...,dnaJ peptide: dnaJP1 was taken in pill form at...,Drug\ndnaJ peptide: dnaJP1 was taken in pill f...
2,13690845,NCT00001337,Experimental,Arm A,EPOCH + Rituximab every 3 weeks for 6 cycles.,13274570,Drug,EPOCH,Combination chemotherapy given with Rituximab ...,Arm A\n\nEPOCH + Rituximab every 3 weeks for 6...,EPOCH: Combination chemotherapy given with Rit...,Drug\nEPOCH: Combination chemotherapy given wi...
3,13690845,NCT00001337,Experimental,Arm A,EPOCH + Rituximab every 3 weeks for 6 cycles.,13274571,Biological,Rituximab,Rituximab given on Day 1 of combination chemot...,Arm A\n\nEPOCH + Rituximab every 3 weeks for 6...,Rituximab: Rituximab given on Day 1 of combina...,Biological\nRituximab: Rituximab given on Day ...
4,13732309,NCT00001563,Experimental,1,EPOCH-R every 3 weeks for up to 6 cycle,13311178,Biological,Filgrastim,Filgrastim after EPOCH-R from Day 6 for 10 day...,1\n\nEPOCH-R every 3 weeks for up to 6 cycle,Filgrastim: Filgrastim after EPOCH-R from Day ...,Biological\nFilgrastim: Filgrastim after EPOCH...


In [4]:
from ctdi_treatment.treatment_pipeline import build_treatment_pipeline, build_df, LightPipeline, PipelineModel

pipeline = build_treatment_pipeline()
p_model = pipeline.fit(spark.createDataFrame([("",)]).toDF("text"))
l_model = LightPipeline(p_model)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]
ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[OK!]
ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[OK!]
ner_clinical_large download started this may take some time.
Approximate size to download 13.9 MB
[OK!]
ner_jsl download started this may take some time.
Approximate size to download 14.5 MB
[OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_rxnorm_disposition download started this may take some time.
Approximate size to

In [5]:
annotation_df = alldf.groupby(["nct_id","design_group_id","group_type","title"]).agg(
    intervention_name_desc=("intervention_name_desc", tit_sep.join),
    arm_title_desc = ('arm_title_desc', tit_sep.join),
    intervention_type = ( 'intervention_type', '|'.join),
#     group_type = ( 'group_type' , tit_sep.join ),
#     title = (  'title', tit_sep.join )
).reset_index()
annotation_df.head(10)

,nct_id,design_group_id,group_type,title,intervention_name_desc,arm_title_desc,intervention_type
0,NCT00000435,13299803,Placebo Comparator,A,None-placebo: placebo was taken in pill form a...,A\n\nSubjects randomized to arm A received 25m...,Drug
1,NCT00000435,13299804,Active Comparator,B,dnaJ peptide: dnaJP1 was taken in pill form at...,B\n\nSubjects randomized to Arm B received 25m...,Drug
2,NCT00001337,13690845,Experimental,Arm A,EPOCH: Combination chemotherapy given with Rit...,Arm A\n\nEPOCH + Rituximab every 3 weeks for 6...,Drug|Biological
3,NCT00001563,13732309,Experimental,1,Filgrastim: Filgrastim after EPOCH-R from Day ...,1\n\nEPOCH-R every 3 weeks for up to 6 cycle\n...,Biological|Biological|Drug
4,NCT00001575,13299883,Experimental,Anti-Tac yttrium 90-labeled humanized anti-Tac...,Y-90 Humanized Anti-Tac: 10 mCi (if a bone mar...,Anti-Tac yttrium 90-labeled humanized anti-Tac...,Biological|Drug
5,NCT00001586,13299880,Experimental,Low-Intermediate Risk B-Cell Pts,Leukemic or stroma cells: Patients are eligibl...,Low-Intermediate Risk B-Cell Pts\n\nPreviously...,Other
6,NCT00001586,13299881,Experimental,Intermediate-high Risk B-Cell Pts,Rituximab: Rituxan.\n\nFludarabine phosphate: ...,Intermediate-high Risk B-Cell Pts\n\nPreviousl...,Biological|Drug
7,NCT00001984,13299801,Experimental,Alemtuzumab and DSG,Alemtuzumab and DSG: Alemtuzumab was administe...,Alemtuzumab and DSG\n\nThe recipients of live ...,Drug
8,NCT00002548,13296605,Active Comparator,HDCTX and PBSC,doxorubicin hydrochloride: 10 mg/m2/day contin...,HDCTX and PBSC\n\nHigh dose chemotherapy with ...,Drug|Drug|Drug|Drug|Procedure
9,NCT00002548,13296606,Experimental,HDCTX with PBSC and Autologous BMT,carmustine: 20 mg/m2 I.V. day 1 q 35 days.\n\n...,HDCTX with PBSC and Autologous BMT\n\nHigh dos...,Drug|Drug|Drug|Drug|Drug|Drug|Procedure|Proced...


In [6]:
intervention_res = l_model.fullAnnotate(annotation_df['intervention_name_desc'].values.tolist())
#arm_res = l_model.fullAnnotate(annotation_df['arm_title_desc'].values.tolist())

In [16]:
intervention_res[0]['resolution_rxnorm']

[Annotation(entity, 0, 11, 1872077, {'all_k_results': '1872077:::220959:::1314253', 'all_k_distances': '7.4137:::8.0947:::8.1481', 'confidence': '0.5035', 'all_k_cosine_distances': '0.0806:::0.0967:::0.0960', 'all_k_resolutions': 'neverpain:::xs no aspirin pr:::nonanal', 'target_text': 'None-placebo', 'all_k_aux_labels': '-:::-:::-', 'token': 'None-placebo', 'resolved_text': 'neverpain', 'all_k_confidences': '0.5035:::0.2549:::0.2416', 'distance': '7.4137', 'sentence': '0'}),
 Annotation(entity, 0, 11, 1872077, {'all_k_results': '1872077:::220959:::1314253', 'all_k_distances': '7.4137:::8.0947:::8.1481', 'confidence': '0.5035', 'all_k_cosine_distances': '0.0806:::0.0967:::0.0960', 'all_k_resolutions': 'neverpain:::xs no aspirin pr:::nonanal', 'target_text': 'None-placebo', 'all_k_aux_labels': '-:::-:::-', 'token': 'None-placebo', 'resolved_text': 'neverpain', 'all_k_confidences': '0.5035:::0.2549:::0.2416', 'distance': '7.4137', 'sentence': '1'}),
 Annotation(entity, 14, 20, 8375, {'al

In [19]:
#p_model.transform(spark.createDataFrame(pd.DataFrame({'text': annotation_df['intervention_name_desc']}))).toPandas()

intervention_res[0]['drug_chunk']


[Annotation(chunk, 0, 11, None-placebo, {'entity': 'Drug', 'confidence': '0.964', 'chunk': '0', 'sentence': '0'}),
 Annotation(chunk, 0, 11, None-placebo, {'entity': 'Drug', 'confidence': '0.964', 'chunk': '1', 'sentence': '0'}),
 Annotation(chunk, 14, 20, placebo, {'entity': 'Drug', 'confidence': '0.9979', 'chunk': '2', 'sentence': '1'}),
 Annotation(chunk, 14, 20, placebo, {'entity': 'Drug', 'confidence': '0.9979', 'chunk': '3', 'sentence': '1'}),
 Annotation(chunk, 35, 38, pill, {'entity': 'Form', 'confidence': '0.9789', 'chunk': '4', 'sentence': '1'}),
 Annotation(chunk, 48, 55, 25mg/day, {'entity': 'Strength', 'confidence': '0.4543', 'chunk': '5', 'sentence': '1'}),
 Annotation(chunk, 57, 68, for 6 months, {'entity': 'CycleLength', 'confidence': '0.84156674', 'chunk': '6', 'sentence': '1'})]

In [10]:
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import math
import pandas as pd


sbert = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli","en","clinical/models")\
            .setInputCols("document").setOutputCol("sbert_embeddings_sbert")

rxnorm = SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm_disposition","en","clinical/models")\
.setInputCols(f"document","sbert_embeddings_sbert").setOutputCol("resolution_rxnorm")#.setNeighbours(3)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_rxnorm_disposition download started this may take some time.
Approximate size to download 804.2 MB
[OK!]


In [34]:
import random as rand
import datetime

def generate_hash(length=10):    
    nums = list(range(48,58))
    uppers = list(range(65,91))
    lowers = list(range(97,123))
    all_chars = nums+uppers+lowers
    return "".join([chr(all_chars[rand.randint(0, len(all_chars)-1)]) for x in range(length)])

def create_import_json (username, title_list, text_list, ner_chunks):
    
    assert len(text_list) == len(ner_chunks)
    
    def build_label(chunk, start, end, label):
        
        id_ = generate_hash()
        
        label_json = {
                "from_name": "label",
                "id": id_,
                "source": "$text",
                "to_name": "text",
                "type": "labels",
                "value": {
                  "end": end,
                  "labels": [label],
                  "start": start,
                  "text": chunk
                }
              }
        return label_json, id_

    import_json = []
    ##important
    index_offset = 0
    
    
    for index, row in enumerate(ner_chunks):
       
        results_jsons = [] 
        
        ent_id_dict = {}
        
        for x in row["full_chunk"]:
            
            b_l, id_ = build_label(x.result, x.begin, x.end+1, x.metadata["entity"].lower())
            
            ent_id_dict[int(x.begin)] = id_
            
            results_jsons.append(b_l)
                
        #         for x in row["assertion"]:

        #             if not pd.isna(x) and x.result.lower() != 'present':

        #                 results_jsons.append(build_label(x.result.title(), x.begin, x.end+1, x.result.title()))

        for x in row['relations']:
            from_ent = int(x.metadata['entity1_begin'])
            to_end = int(x.metadata['entity2_begin'])
            direction = "right"
            results_jsons.append({
                "from_id": ent_id_dict[from_ent],
                "to_id": ent_id_dict[to_end],
                "type": "relation",
                "direction": "right",
                "labels": []
            })
        
        import_json.append({"predictions": [{
            'created_username': username,
                "result":results_jsons
            }],
            "data":{
                "text":text_list[index],
                "title":title_list[index]
            },
                            'id': index + index_offset
                           })
    
    print ("Annotations payload is ready")

    
    return import_json #saved_json

In [23]:
annotation_df['u_key'] = annotation_df['nct_id'].astype('str') + annotation_df['design_group_id'].astype('str')
annotation_df['u_key'].nunique()

13

In [35]:
pre_ann_json = create_import_json (username = 'pipelie_with_trtment', 
                    title_list = annotation_df['u_key'].values.tolist(),
                    text_list = annotation_df['intervention_name_desc'].values.tolist(), 
                    ner_chunks = intervention_res)

print (len(pre_ann_json))

Annotations payload is ready
13


In [36]:
with open('pre_annotation.json', 'w') as f_:
    f_.write(json.dumps(pre_ann_json, indent=4))